In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.utils.data as Data
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from PIL import Image
import random

from sklearn.metrics import accuracy_score
from copy import deepcopy

## Read in training and testing data

In [116]:
training_seq = np.load("../data/mlData/training_seqData.npz")
training_dna = training_seq["dna"].astype(np.int8)
training_dna_labels = training_seq["label"].astype(np.int8)
# training_dna_labels = np.where(training_dna_labels == [2], [0], [1])

testing_seq = np.load("../data/mlData/testing_seqData.npz")
testing_dna = testing_seq["dna"].astype(np.int8)
testing_dna_labels = testing_seq["label"].astype(np.int8)
# testing_dna_labels = np.where(testing_dna_labels == [2], [0], [1])


In [117]:
print(training_dna.shape)
print(training_dna_labels.shape)
print(testing_dna.shape)
print(testing_dna_labels.shape)

(2194, 4, 2200)
(2194, 1)
(130, 4, 2200)
(130, 1)


In [118]:
chip_seq = np.load("../data/mlData/training_chipData.npz")
training_chip = chip_seq["chip"].astype(np.float32)
training_chip_labels = chip_seq["label"].astype(np.int8)

chip_seq = np.load("../data/mlData/testing_chipData.npz")
testing_chip = chip_seq["chip"].astype(np.float32)
testing_chip_labels = chip_seq["label"].astype(np.int8)


In [119]:
print(training_chip.shape)
print(training_chip_labels.shape)
print(testing_chip.shape)
print(testing_chip_labels.shape)

(2194, 20, 2200)
(2194, 1)
(130, 20, 2200)
(130, 1)


In [120]:
train_dataloader = DataLoader(training_seq, batch_size=10, shuffle=False)
for dna in iter(train_dataloader):
    print(dna)
    print(labels)

KeyError: '0 is not a file in the archive'

## Build CNN

In [105]:
class Pol3_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        

        """
        Define your layers here.
        """
        
        self.conv1 = nn.Conv1d(in_channels=4,out_channels=320,kernel_size=4)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=4)
        self.bn1 = nn.BatchNorm1d(320)
        self.conv2 = nn.Conv1d(in_channels=320, out_channels=480, kernel_size=4)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=4)
        self.bn2 = nn.BatchNorm1d(480)
        self.output_layer = nn.Linear(65280, 1)


    def forward(self, X):

        """
        Design the process of your network.
        """
        
        X = self.conv1(X)
        X = self.bn1(X)
        X = self.relu1(X)
        X = self.maxpool1(X)
        
        X = self.conv2(X)
        X = self.bn2(X)
        X = self.relu2(X)
        X = self.maxpool2(X)
        
        X = torch.flatten(X, 1)        

        logits = self.output_layer(X)
        return logits   # do not apply softmax

    def classify(self, X):
        
        """
        Write a function that outputs the labels.
        """
        logits = self(X)
        logits = F.softmax(logits, dim=1)
        logits = nn.functional.normalize(logits, dim = 1)
        print("logits_class", logits)
        labels = torch.argmax(logits, dim=1)
        print("labels", labels)
        return labels.type(torch.long)

In [106]:
def train(model, training_data, training_labels, testing_data, testing_labels, epochs=15, batch_size=16, lr=1e-3):
    """
    Q:  write the training loop following the schema shown above.

    Inputs
    - model: the model to be trained - a PyTorch nn.Module class object
    - X_train, y_train, X_val, y_val: training and validation data
    - epochs: num epochs, or the number of times we want to run through the entire training data
    - batch_size: number of data points per batch
    - lr: learning rate
    - optimizer: optimizer used

    Outputs
    - losses: a list of losses
    - accuracies: a list of accuracies
    """

    torch.cuda.empty_cache()

    best_acc = -1
    best_model = None

    batches = int(np.ceil(len(training_data) / batch_size))

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    losses = []
    accuracies = []

    testing_data = torch.tensor(testing_data).type(torch.float32)
    testing_labels = torch.tensor(testing_labels).type(torch.float32)

    for epoch in range(epochs):
        for i in range(batches):
#             if i % 1000 == 0:
#               print("Epoch " + str(epoch+1) + "/" + str(epochs) + ": " + str(i/batches*100) + "%")
            X_batch = training_data[i*batch_size:i*batch_size+batch_size]
            X_batch = torch.tensor(X_batch).type(torch.float32)
            y_batch = training_labels[i*batch_size:i*batch_size+batch_size]
            y_batch = torch.tensor(y_batch).type(torch.int8)
            
#             print(y_batch)
            
            logits = model(X_batch)
#             print("logits", logits)
#             print("y_batch", y_batch)
            loss = loss_fn(logits, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # calculate the validation accuracy and append the loss of this epoch
        y_pred_test = model.classify(testing_data)
        accuracy = accuracy_score(testing_labels.cpu(), y_pred_test.cpu())
        accuracies.append(accuracy)
        losses.append(loss)

        if accuracy > best_acc:
          best_acc = accuracy
          best_model = deepcopy(model)

        # print epoch, loss, and current test accuracy (don't delete this line - it's slightly more organized now)
        print(f"Epoch {epoch + 1}:\tloss {np.round(loss.detach().cpu().numpy().item(), 4)}\t& accuracy {np.round(accuracy, 4)}")
    print(f"Resetting model... Best validation accuracy:\t{np.round(best_acc, 4)}")
    model.load_state_dict(best_model.state_dict())
    return losses, accuracies


In [107]:
model = Pol3_CNN()
losses, accuracies = train(model, training_dna, training_dna_labels, testing_dna, testing_dna_labels, batch_size=10, epochs=10)


logits_class tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
       

KeyboardInterrupt: 